# Create an InterActive Graph of NTRL Lab Analysis Results

In [4]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from dateutil.parser import parse
from itertools import compress
from collections import OrderedDict
from datetime import date
import datetime
import re
from datetime import timedelta
import openpyxl
from sklearn.metrics import mean_squared_error
import plotly.express as px



In [5]:
#used in the analysis results data. cleans up the sampling time entry
def time_cleaner(x):
    time_regex = re.compile(r'[0-8a-zA-Z:]', re.IGNORECASE| re.VERBOSE|re.DOTALL)
    timeact = time_regex.findall(x)
    newtime = ''.join(timeact)
    t = parse(x)
    new_str_time = str(t.hour)+':'+str(t.minute)+':'+str(t.second)
    return new_str_time
def ave_filler(dfname, colname):
    dfcn = dfname[colname]
    try:        
        dfcn = pd.to_numeric(dfcn, errors='coerce')
    except:
        print('error found in' + str(colname))
        pass
    try:
        dfname[colname] = dfcn.where(dfcn.notnull(), other=(dfcn.fillna(method='ffill') + dfcn.fillna(method='bfill'))/2)
    except:
        print('nan not fixed' + str(colname))   
    return dfname
def generate_df_from_analysis(workbookTab, col_sampleDate, col_sampleTime, col_analysis, col_title):
    #get the start and end sample dates
    Smpl_day_index = list()
    Chem_Element_datelist = list()
    Chem_Element_list = list()
    Smpl_day_index = list()
    Chem_Element_datelist = list()
    Chem_Element_list = list()
    plus_oneday_list = ['0:0:0', '1:0:0', '2:0:0', '3:0:0', '4:0:0', '5:0:0']
    #gets the date day from the B column of the analysis file if there is value in the cell
    for i in range(1, workbookTab.max_row):
        if type(workbookTab[str(col_sampleDate) + str(i)].value) == int or type(workbookTab[str(col_sampleDate) + str(i)].value) == float:
            Smpl_day_index.append(i)
    #get the ending row that has date day
    for j in range(1, workbookTab.max_row):
        if type(workbookTab[str(col_sampleTime) + str(j)].value) == datetime.datetime:
            smpl_end_index = j
    for y in range(0, len(Smpl_day_index)):
        if Smpl_day_index[y] != Smpl_day_index[-1]:      
            for x in range(Smpl_day_index[y], Smpl_day_index[y+1]):
                if workbookTab[str(col_analysis)+str(x)].value is None: 
                    pass
                else:
                    if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                        date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))  
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)
                    else:
                        date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))  
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)

        if Smpl_day_index[y] == Smpl_day_index[-1]:
            for x in range(Smpl_day_index[y], smpl_end_index+1):
                if workbookTab[str(col_analysis)+str(x)].value is None:   
                    pass
                else:
                    if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                        date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)
                    else:
                        date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)

    Chem_Element_list_pair = list(zip(Chem_Element_list, Chem_Element_datelist[0:]))
    Chem_Element_df = pd.DataFrame(Chem_Element_list_pair)
    Chem_Element_df.columns = [col_title, 'DATETIME'] ##check if list title is updated
    Chem_Element_df.set_index('DATETIME', inplace=True)

    return Chem_Element_df

def generate_add_trace(figname, df_elemental, axis_bool):
    colname = df_elemental.columns[0]
    figname.add_trace(go.Scatter(x=df_elemental.index, y=df_elemental[colname], name=colname), secondary_y=axis_bool,)
#---------------------------------------

class LabResults():
    def __init__(self, monthyear, filePath):
        #____monthyear format examle is datetime.date(2020,7,1)
        #____filepath example is '\\thgobna001\userdata\THPAL\...\ANALYSIS RESULTS 2020\7) July _2020.xlsx'
        #____add r' at the start of the filepath to apply the regex that allows the use of the filepath as raw
        #____sef.workbook is the uploaded excel workbook of the entire analysis files so it takes time to load
        #____Create an instance of the workbook by giving it a varible name with the sample format: Jan_2020
        
        import openpyxl
        import io
        with open(filePath, "rb") as f:
            in_mem_file = io.BytesIO(f.read())
        
        self.workbook = openpyxl.load_workbook(in_mem_file, data_only=True)
        self.monthyear = monthyear   
        
    def generate_df_from_analysis(self, tabname, list_lab_index):
        #____tabname is a string type name of the tab in the laboratory analysis excel file.
        #____-------Example tabnames: '104PU01', '105TK03', '106TK01'
        #____-------Inside these tabs are the specific analysis results done on the sample taken from the 'tabname' area .
        #____list_lab_index is a list with the sample format is: ['B', 'C', 'E', '104_Pb']
        #____-------the first letter 'B' is the column letter where the day of the month is written as integers 1,2,3..30
        #____-------the second letter 'C' is the column letter where the sampling time is written; e.g. '7:00:00 AM'
        #____-------the third letter 'E' is the column where the results of the specific analysis are listed
        #____-------the last item in the list is the string you'll use as the new title; e.g. '104_Pb'
        
        workbookTab = self.workbook[tabname]
        self.list_lab_index = list_lab_index
        col_sampleDate = self.list_lab_index[0]
        col_sampleTime = self.list_lab_index[1]
        col_analysis = self.list_lab_index[2]
        col_title = self.list_lab_index[3]
        currentmonthyr = self.monthyear
        
        #get the start and end sample dates
        Smpl_day_index = list()
        Chem_Element_datelist = list()
        Chem_Element_list = list()
        Smpl_day_index = list()
        Chem_Element_datelist = list()
        Chem_Element_list = list()
        plus_oneday_list = ['0:0:0', '1:0:0', '2:0:0', '3:0:0', '4:0:0', '5:0:0']
        #gets the date day from the B column of the analysis file if there is value in the cell
        for i in range(1, workbookTab.max_row):
            if type(workbookTab[str(col_sampleDate) + str(i)].value) == int or type(workbookTab[str(col_sampleDate) + str(i)].value) == float:
                Smpl_day_index.append(i)
        #get the ending row that has date day
        for j in range(1, workbookTab.max_row):
            if type(workbookTab[str(col_sampleTime) + str(j)].value) == datetime.datetime:
                smpl_end_index = j
        for y in range(0, len(Smpl_day_index)):
            if Smpl_day_index[y] != Smpl_day_index[-1]:      
                for x in range(Smpl_day_index[y], Smpl_day_index[y+1]):
                    if workbookTab[str(col_analysis)+str(x)].value is None: 
                        pass
                    else:
                        try:
                            if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                                date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                                +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                                Chem_Element_datelist.append(parse(date))  
                                chem_element = workbookTab[str(col_analysis)+str(x)].value
                                Chem_Element_list.append(chem_element)
                            else:
                                date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                                +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                                Chem_Element_datelist.append(parse(date))  
                                chem_element = workbookTab[str(col_analysis)+str(x)].value
                                Chem_Element_list.append(chem_element)
                        except:
                            pass

            if Smpl_day_index[y] == Smpl_day_index[-1]:
                for x in range(Smpl_day_index[y], smpl_end_index+1):
                    if workbookTab[str(col_analysis)+str(x)].value is None:   
                        pass
                    else:
                        try:
                            if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                                date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                                +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                                Chem_Element_datelist.append(parse(date))
                                chem_element = workbookTab[str(col_analysis)+str(x)].value
                                Chem_Element_list.append(chem_element)
                            else:
                                date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                                +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                                Chem_Element_datelist.append(parse(date))
                                chem_element = workbookTab[str(col_analysis)+str(x)].value
                                Chem_Element_list.append(chem_element)
                        except:
                            pass

        Chem_Element_list_pair = list(zip(Chem_Element_list, Chem_Element_datelist[0:]))
        Chem_Element_df = pd.DataFrame(Chem_Element_list_pair)
        Chem_Element_df.columns = [col_title, 'DATETIME'] ##check if list title is updated
        Chem_Element_df.set_index('DATETIME', inplace=True)

        return Chem_Element_df
    
def join_DF(dfsList1):
    df = dfsList1[0]
    for i in range(1,len(dfsList1)):
        df = df.join(dfsList1[i], how='outer')
    return df

def append_monthsDF(dfsListmonthly): #takes the list of dataframes from monthly lab results of one sampling area
    df = dfsListmonthly[0]
    for i in range(1,len(dfsListmonthly)):
        df = df.append(dfsListmonthly[i], ignore_index=True)
    return df    



#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from dateutil.parser import parse
from itertools import compress
from collections import OrderedDict
from datetime import date
import datetime
import re
from datetime import timedelta
import openpyxl
#from sklearn.metrics import mean_squared_error
import plotly.express as px

import plotly.graph_objects as go
from plotly.subplots import make_subplots
# Create figure with secondary y-axis

#used in the analysis results data. cleans up the sampling time entry
def time_cleaner(x):
    time_regex = re.compile(r'[0-8a-zA-Z:]', re.IGNORECASE| re.VERBOSE|re.DOTALL)
    timeact = time_regex.findall(x)
    newtime = ''.join(timeact)
    t = parse(x)
    new_str_time = str(t.hour)+':'+str(t.minute)+':'+str(t.second)
    return new_str_time
def ave_filler(dfname, colname):
    dfcn = dfname[colname]
    try:        
        dfcn = pd.to_numeric(dfcn, errors='coerce')
    except:
        print('error found in' + str(colname))
        pass
    try:
        dfname[colname] = dfcn.where(dfcn.notnull(), other=(dfcn.fillna(method='ffill') + dfcn.fillna(method='bfill'))/2)
    except:
        print('nan not fixed' + str(colname))   
    return dfname
def generate_df_from_analysis(workbookTab, col_sampleDate, col_sampleTime, col_analysis, col_title):
    #get the start and end sample dates
    Smpl_day_index = list()
    Chem_Element_datelist = list()
    Chem_Element_list = list()
    Smpl_day_index = list()
    Chem_Element_datelist = list()
    Chem_Element_list = list()
    plus_oneday_list = ['0:0:0', '1:0:0', '2:0:0', '3:0:0', '4:0:0', '5:0:0']
    #gets the date day from the B column of the analysis file if there is value in the cell
    for i in range(1, workbookTab.max_row):
        if type(workbookTab[str(col_sampleDate) + str(i)].value) == int or type(workbookTab[str(col_sampleDate) + str(i)].value) == float:
            Smpl_day_index.append(i)
    #get the ending row that has date day
    for j in range(1, workbookTab.max_row):
        if type(workbookTab[str(col_sampleTime) + str(j)].value) == datetime.datetime:
            smpl_end_index = j
    for y in range(0, len(Smpl_day_index)):
        if Smpl_day_index[y] != Smpl_day_index[-1]:      
            for x in range(Smpl_day_index[y], Smpl_day_index[y+1]):
                if workbookTab[str(col_analysis)+str(x)].value is None: 
                    pass
                else:
                    if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                        date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))  
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)
                    else:
                        date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))  
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)

        if Smpl_day_index[y] == Smpl_day_index[-1]:
            for x in range(Smpl_day_index[y], smpl_end_index+1):
                if workbookTab[str(col_analysis)+str(x)].value is None:   
                    pass
                else:
                    if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                        date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)
                    else:
                        date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)

    Chem_Element_list_pair = list(zip(Chem_Element_list, Chem_Element_datelist[0:]))
    Chem_Element_df = pd.DataFrame(Chem_Element_list_pair)
    Chem_Element_df.columns = [col_title, 'DATETIME'] ##check if list title is updated
    Chem_Element_df.set_index('DATETIME', inplace=True)

    return Chem_Element_df

def generate_add_trace(figname, df_elemental, axis_bool):
    colname = df_elemental.columns[0]
    figname.add_trace(go.Scatter(x=df_elemental.index, y=df_elemental[colname], name=colname), secondary_y=axis_bool,)
#---------------------------------------

#--------------------------------------------------End-------------------------------------------------------------------
#
#
def generate_plot_from_df(figname, df,df_axis_truefalse_list):
    for i in range(0, len(df.columns)):
        colname = df.columns[i]
        figname.add_trace(go.Scatter(x=df[colname].index, y=df[colname], name=colname), secondary_y=df_axis_truefalse_list[i],)
#
#
#--------------------------------------------------------------End-----------------------------------------------------------------------

def generate_add_trace(figname, df_elemental, axis_bool):
    colname = df_elemental.columns[0]
    figname.add_trace(go.Scatter(x=df_elemental.index, y=df_elemental[colname], name=colname), secondary_y=axis_bool,)


In [6]:
currentmonthyr_08_2020 = datetime.date(2020,8,1)
currentmonthyr_09_2020 = datetime.date(2020,9,1)

#filepath

filepath_08_2020 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\08 AUGUST 2020\8) August _2020.xlsx'
filepath_09_2020 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\9) September _2020.xlsx'

In [7]:
#define the filepaths and the monthyr data for year 2019
currentmonthyr_09_2020 = datetime.date(2020,9,1)


#filepath
filepath_09_2020 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\9) September _2020.xlsx'

ntrl_Pb_3 = ['B', 'C', 'E', '104_Pb']
ntrl_Zn_3 = ['B', 'C', 'F', '104_Zn']
ntrl_Cu_3 = ['B', 'C', 'G', '104_Cu']
ntrl_Ni_3 = ['B', 'C', 'H', '104_Ni']
ntrl_Co_3 = ['B', 'C', 'I', '104_Co']
ntrl_Fe_3 = ['B', 'C', 'J', '104_Fe']
ntrl_Mn_3 = ['B', 'C', 'K', '104_Mn']
ntrl_Cr_3 = ['B', 'C', 'L', '104_Cr']
ntrl_Ca_3 = ['B', 'C', 'M', '104_Ca']
ntrl_Si_3 = ['B', 'C', 'N', '104_Si']
ntrl_Al_3 = ['B', 'C', 'O', '104_Al']
ntrl_Mg_3 = ['B', 'C', 'P', '104_Mg']
ntrl_Fe2_3 = ['B', 'C', 'R', '104_Fe2']
ntrl_pH60_3 = ['B', 'C', 'BB', '104_pH60'] # check
ntrl_ORP_3 = ['B', 'C', 'AJ', '104_ORP'] 
ntrl_NTU_3 = ['B', 'C', 'AN', '104_NTU'] # check
ntrl_Ft_3 = ['B', 'C', 'AW', '104_Ft'] #check

tk13_Zn_3 = ['B', 'C', 'E', 'tk13_Zn']
tk13_Solids_3 = ['B', 'C', 'AZ', 'tk13_solids%'] #check

MStk01_Zn_drop_3 = ['B', 'C', 'BD', 'Zn_drop']
MStk01_Zn_3 = ['B', 'C', 'F', '106_Zn']
MStk01_Ni_3 = ['B', 'C', 'H', '106_Ni']


tk05_Pb = ['B', 'C', 'E', 'tk05_Pb']
tk05_Zn = ['B', 'C', 'F', 'tk05_Zn']
tk05_Cu = ['B', 'C', 'G', 'tk05_Cu']
tk05_Ni = ['B', 'C', 'H', 'tk05_Ni']
tk05_Co = ['B', 'C', 'I', 'tk05_Co']
tk05_Fe = ['B', 'C', 'J', 'tk05_Fe']
tk05_Mn = ['B', 'C', 'K', 'tk05_Mn']
tk05_Cr = ['B', 'C', 'L', 'tk05_Cr']
tk05_Ca = ['B', 'C', 'M', 'tk05_Ca']
tk05_Si = ['B', 'C', 'N', 'tk05_Si']
tk05_Al = ['B', 'C', 'O', 'tk05_Al']
tk05_Mg = ['B', 'C', 'P', 'tk05_Mg']
 
monthLab_3 = LabResults(currentmonthyr_09_2020, filepath_09_2020)
df_month_3 = monthLab_3.generate_df_from_analysis('104PU01', ntrl_Pb_3).join(
    monthLab_3.generate_df_from_analysis('104PU01', ntrl_Zn_3), how='outer').join(
    monthLab_3.generate_df_from_analysis('104PU01', ntrl_Cu_3), how='outer').join(
    monthLab_3.generate_df_from_analysis('104PU01', ntrl_Ni_3), how='outer').join(
    monthLab_3.generate_df_from_analysis('104PU01', ntrl_Co_3), how='outer').join(
    monthLab_3.generate_df_from_analysis('104PU01', ntrl_Fe_3), how='outer').join(
    monthLab_3.generate_df_from_analysis('104PU01', ntrl_Mn_3), how='outer').join(
    monthLab_3.generate_df_from_analysis('104PU01', ntrl_Cr_3), how='outer').join(
    monthLab_3.generate_df_from_analysis('104PU01', ntrl_Ca_3), how='outer').join(
    monthLab_3.generate_df_from_analysis('104PU01', ntrl_Si_3), how='outer').join(
    monthLab_3.generate_df_from_analysis('104PU01', ntrl_Al_3), how='outer').join(
    monthLab_3.generate_df_from_analysis('104PU01', ntrl_Mg_3), how='outer').join(
    monthLab_3.generate_df_from_analysis('104PU01', ntrl_Fe2_3), how='outer').join(
    monthLab_3.generate_df_from_analysis('104PU01', ntrl_pH60_3), how='outer').join(
    monthLab_3.generate_df_from_analysis('104PU01', ntrl_ORP_3), how='outer').join(
    monthLab_3.generate_df_from_analysis('104PU01', ntrl_NTU_3), how='outer').join(
    monthLab_3.generate_df_from_analysis('104PU01', ntrl_Ft_3), how='outer').join(
    monthLab_3.generate_df_from_analysis('105TK13', tk13_Zn_3), how='outer').join(
    monthLab_3.generate_df_from_analysis('105TK13', tk13_Solids_3), how='outer').join(
    monthLab_3.generate_df_from_analysis('106TK01', MStk01_Zn_drop_3), how='outer').join(
    monthLab_3.generate_df_from_analysis('106TK01', MStk01_Zn_3), how='outer').join(
    monthLab_3.generate_df_from_analysis('106TK01', MStk01_Ni_3), how='outer').join(
    monthLab_3.generate_df_from_analysis('105TK05', tk05_Pb).join(
        monthLab_3.generate_df_from_analysis('105TK05', tk05_Zn), how='outer').join(
        monthLab_3.generate_df_from_analysis('105TK05', tk05_Cu), how='outer').join(
        monthLab_3.generate_df_from_analysis('105TK05', tk05_Ni), how='outer').join(
        monthLab_3.generate_df_from_analysis('105TK05', tk05_Co), how='outer').join(
        monthLab_3.generate_df_from_analysis('105TK05', tk05_Fe), how='outer').join(
        monthLab_3.generate_df_from_analysis('105TK05', tk05_Mn), how='outer').join(
        monthLab_3.generate_df_from_analysis('105TK05', tk05_Cr), how='outer').join(
        monthLab_3.generate_df_from_analysis('105TK05', tk05_Ca), how='outer').join(
        monthLab_3.generate_df_from_analysis('105TK05', tk05_Si), how='outer').join(
        monthLab_3.generate_df_from_analysis('105TK05', tk05_Al), how='outer').join(
        monthLab_3.generate_df_from_analysis('105TK05', tk05_Mg), how='outer'))

for i in df_month_3.columns:
    df_month_3[i] = pd.to_numeric(df_month_3[i], errors='coerce')

df_month_3 = df_month_3.interpolate(method='linear', limit_direction='forward', axis=0)
df_month_3['Zn_drop_adj'] = df_month_3['Zn_drop']*100


#------------------------------------------------------------------------------------------extracting data from the August 2020 dataset-__start__-----------------------------------------------------------------------
monthLab_2 = LabResults(currentmonthyr_08_2020, filepath_08_2020) 
df_month_2 = monthLab_2.generate_df_from_analysis('104PU01', ntrl_Pb_3).join(
    monthLab_2.generate_df_from_analysis('104PU01', ntrl_Zn_3), how='outer').join(
    monthLab_2.generate_df_from_analysis('104PU01', ntrl_Cu_3), how='outer').join(
    monthLab_2.generate_df_from_analysis('104PU01', ntrl_Ni_3), how='outer').join(
    monthLab_2.generate_df_from_analysis('104PU01', ntrl_Co_3), how='outer').join(
    monthLab_2.generate_df_from_analysis('104PU01', ntrl_Fe_3), how='outer').join(
    monthLab_2.generate_df_from_analysis('104PU01', ntrl_Mn_3), how='outer').join(
    monthLab_2.generate_df_from_analysis('104PU01', ntrl_Cr_3), how='outer').join(
    monthLab_2.generate_df_from_analysis('104PU01', ntrl_Ca_3), how='outer').join(
    monthLab_2.generate_df_from_analysis('104PU01', ntrl_Si_3), how='outer').join(
    monthLab_2.generate_df_from_analysis('104PU01', ntrl_Al_3), how='outer').join(
    monthLab_2.generate_df_from_analysis('104PU01', ntrl_Mg_3), how='outer').join(
    monthLab_2.generate_df_from_analysis('104PU01', ntrl_Fe2_3), how='outer').join(
    monthLab_2.generate_df_from_analysis('104PU01', ntrl_pH60_3), how='outer').join(
    monthLab_2.generate_df_from_analysis('104PU01', ntrl_ORP_3), how='outer').join(
    monthLab_2.generate_df_from_analysis('104PU01', ntrl_NTU_3), how='outer').join(
    monthLab_2.generate_df_from_analysis('104PU01', ntrl_Ft_3), how='outer').join(
    monthLab_2.generate_df_from_analysis('105TK13', tk13_Zn_3), how='outer').join(
    monthLab_2.generate_df_from_analysis('105TK13', tk13_Solids_3), how='outer').join(
    monthLab_2.generate_df_from_analysis('106TK01', MStk01_Zn_drop_3), how='outer').join(
    monthLab_2.generate_df_from_analysis('106TK01', MStk01_Zn_3), how='outer').join(
    monthLab_2.generate_df_from_analysis('106TK01', MStk01_Ni_3), how='outer').join(
    monthLab_2.generate_df_from_analysis('105TK05', tk05_Pb).join(
        monthLab_2.generate_df_from_analysis('105TK05', tk05_Zn), how='outer').join(
        monthLab_2.generate_df_from_analysis('105TK05', tk05_Cu), how='outer').join(
        monthLab_2.generate_df_from_analysis('105TK05', tk05_Ni), how='outer').join(
        monthLab_2.generate_df_from_analysis('105TK05', tk05_Co), how='outer').join(
        monthLab_2.generate_df_from_analysis('105TK05', tk05_Fe), how='outer').join(
        monthLab_2.generate_df_from_analysis('105TK05', tk05_Mn), how='outer').join(
        monthLab_2.generate_df_from_analysis('105TK05', tk05_Cr), how='outer').join(
        monthLab_2.generate_df_from_analysis('105TK05', tk05_Ca), how='outer').join(
        monthLab_2.generate_df_from_analysis('105TK05', tk05_Si), how='outer').join(
        monthLab_2.generate_df_from_analysis('105TK05', tk05_Al), how='outer').join(
        monthLab_2.generate_df_from_analysis('105TK05', tk05_Mg), how='outer'))
#------------------------------------------------------------------------------------------extracting data from the August 2020 dataset-__end__-----------------------------------------------------------------------

for i in df_month_2.columns:
    df_month_2[i] = pd.to_numeric(df_month_2[i], errors='coerce')

df_month_2 = df_month_2.interpolate(method='linear', limit_direction='forward', axis=0)
df_month_2['Zn_drop_adj'] = df_month_2['Zn_drop']*100

df_month2_3 = df_month_2.append(df_month_3, sort=True) #-----------combines the data for August2020 with the current available data for September2020


In [8]:
#-------------------------------------------------------------------------------------------------------------------------#
#---------------------Code for predicting the Zn drop value in 106TK01 by applying the coefficients  from RidgeRegression----------#

#----Importing the Pi data for MS flow, rec gas, feed temp
df_MSPi = pd.read_excel(r'C:\Users\v.t.flores\Documents\ZnDrop_Project_Pi_Sept2020.xlsx', sheet_name='PI_sept2020', index_col=False)

#------------------Drop the row[0]. This is the title tag use in the pi datalink. We don't need it in pandas df.
#------------------What remain are the column title and the rest of the rows of data.

df_MSPi = df_MSPi.drop(df_MSPi.index[0])

#Step2: Parse the DATETIME column
df_MSPi['DATETIME'] = df_MSPi['DATETIME'].apply(lambda x: parse(str(x)))
df_MSPi.set_index('DATETIME', inplace=True)


# Step3: Convert each column to numeric to handle nan values and other comments like 'No data...'. Convering to numeric
#------ converts them into a NaN value that pandas understands.
for i in df_MSPi.columns:
    df_MSPi[i] = pd.to_numeric(df_MSPi[i], errors='coerce')

#Creating a column for the correct H2S flowrate
df_MSPi['H2S_tk01'] = np.where(df_MSPi['H2S_FT_TK01_A']>df_MSPi['H2S_FT_TK01_B'],\
                                    df_MSPi['H2S_FT_TK01_A'], df_MSPi['H2S_FT_TK01_B'])

df_MSPi['H2S_tk02'] = np.where(df_MSPi['H2S_FT_TK02_A']>df_MSPi['H2S_FT_TK02_B'],\
                                    df_MSPi['H2S_FT_TK02_A'], df_MSPi['H2S_FT_TK02_B'])    

DF_labdata_and_Pi_Sept2020 = df_month2_3.join(df_MSPi, how='outer')

DF_labdata_and_Pi_Sept2020_ntrpltd = DF_labdata_and_Pi_Sept2020.interpolate(method='linear', limit_direction='both', axis=0)

In [9]:
#REFER to the ZnDrop_Project_RidgeReg.ipynb file on how the df_reg_coef_intercepts_zndropPred.xlsx was calculated
df_reg_coef_intercepts_zndropPred = pd.read_excel(r'C:\Users\v.t.flores\Documents\df_reg_coef_intercepts_zndropPred.xlsx', index_col=False)
#
#
df_reg_coef_intercepts_zndropPred.drop('Unnamed: 0', axis=1, inplace=True)
reg_intercept = df_reg_coef_intercepts_zndropPred['intercept'][0]

DF = DF_labdata_and_Pi_Sept2020_ntrpltd.dropna()


In [11]:
#------------------------------------------------------------------------------------------------------------- Code block for tensorflow model------------------------------------------------------------------------

#model.load(r'C:\Users\v.t.flores\Documents\Zn_drop_TF_model')
import pathlib
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

new_model3 = tf.keras.models.load_model(r'C:\Users\v.t.flores\Documents\Zn_drop_TF_model3')
new_model2 = tf.keras.models.load_model(r'C:\Users\v.t.flores\Documents\Zn_drop_TF_model2')
new_model4 = tf.keras.models.load_model(r'C:\Users\v.t.flores\Documents\Zn_drop_TF_model4')

tk01_Zn_model = tf.keras.models.load_model(r'C:\Users\v.t.flores\Documents\Zn_TF_model1')
tk01_Ni_model = tf.keras.models.load_model(r'C:\Users\v.t.flores\Documents\Ni_TF_model1')

# slice the DF for the columns that will be fed into the tensorflow model

DF_TF = DF[['104_Al', '104_Ca', '104_Co', '104_Cr', '104_Cu', '104_Fe', '104_Fe2',
       '104_Ft', '104_Mg', '104_Mn', '104_NTU', '104_Ni', '104_ORP', '104_Pb',
       '104_Si', '104_Zn', '104_pH60', 'DeZn_Feed_Flow', 'DeZn_Feed_T',
       'DeZn_Seed_Flow', 'DeZn_TK05_T',
       'H2S_tk01', 'H2S_tk02', 'tk05_Al',
       'tk05_Ca', 'tk05_Co', 'tk05_Cr', 'tk05_Cu', 'tk05_Fe', 'tk05_Mg',
       'tk05_Mn', 'tk05_Ni', 'tk05_Pb', 'tk05_Si', 'tk05_Zn']]


# look at the overall statistics:
new_data_stats = DF_TF.describe()
new_data_stats = new_data_stats.transpose()

def norm(x):
    return (x - new_data_stats['mean']) / new_data_stats['std']
normed_new_data = norm(DF_TF)

#predictions2 = new_model2.predict(normed_new_data)
#predictions3 = new_model3.predict(normed_new_data)
predictions4 = new_model4.predict(normed_new_data)
ZN_predictions = tk01_Zn_model.predict(normed_new_data)
NI_predictions = tk01_Ni_model.predict(normed_new_data)


#DF_TF['TF_pred2'] = predictions2
#DF_TF['TF_pred3'] = predictions3
DF_TF['TF_pred4'] = predictions4

DF_TF['MSTK01_Zn_Pred'] = ZN_predictions
DF_TF['MSTK01_Ni_Pred'] = NI_predictions



ValueError: Error when checking input: expected dense_3_input to have shape (34,) but got array with shape (35,)

In [ ]:
# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Create the dataframe for DeZn_TK05 Temp v. Zn Drop

df_test_columns_Include = ['104_Al', '104_Ca', '104_Co', '104_Cr', '104_Cu', '104_Fe', '104_Fe2',
       '104_Ft', '104_Mg', '104_Mn', '104_NTU', '104_Ni', '104_ORP', '104_Pb',
       '104_Si', '104_Zn', '104_pH60', 'DeZn_Feed_Flow', 'DeZn_Feed_T',
       'DeZn_Seed_Flow', 'DeZn_TK05_T', 
       'H2S_tk01', 'H2S_tk02', 'tk05_Al',
       'tk05_Ca', 'tk05_Co', 'tk05_Cr', 'tk05_Cu', 'tk05_Fe', 'tk05_Mg',
       'tk05_Mn', 'tk05_Ni', 'tk05_Pb', 'tk05_Si', 'tk05_Zn']

DF_test2 = DF_TF[df_test_columns_Include].dropna()[-1:] # DF_test2 is the last non NA value in the dataframe

#DF_test2 = DF_TF[df_test_columns_Include][757:758]

temp_range_start = round((DF_test2['DeZn_TK05_T'][0]-10), 0)
temp_range_end = round((DF_test2['DeZn_TK05_T'][0]+10), 0)
temp_range = np.linspace(temp_range_start, temp_range_end, 20)

zn_drop_pred = []
x_list = []
for temp in temp_range:
    DF_test2['DeZn_TK05_T'] = temp
    normed_DF_test2 = norm(DF_test2)
    predictions2 = new_model2.predict(normed_DF_test2)
    zn_drop_pred.append(predictions2[0][0])
    x_list.append(temp)
    
df_tk05temp_v_zndrop = pd.DataFrame(zip(x_list, zn_drop_pred), columns=['DeZn_TK05_T', 'Zn_drop'])
df_tk05temp_v_zndrop = df_tk05temp_v_zndrop.set_index('DeZn_TK05_T')

# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Create the dataframe for H2S_tk01 v. Zn drop

gas_tk01_start = round(DF_test2['H2S_tk01'][0]-10, 2)
gas_tk01_end = round(DF_test2['H2S_tk01'][0]+10, 2)
gas_1_range = np.linspace(gas_tk01_start, gas_tk01_end, 20)

zn_drop_pred_h2s1 = []
x_list_h2s1 = []
for gas1 in gas_1_range:
    DF_test2['H2S_tk01'] = gas1
    normed_DF_test2 = norm(DF_test2)
    predictions2 = new_model2.predict(normed_DF_test2)
    zn_drop_pred_h2s1.append(predictions2[0][0])
    x_list_h2s1.append(gas1)
    
df_H2STK01_v_zndrop = pd.DataFrame(zip(x_list_h2s1, zn_drop_pred_h2s1), columns=['H2S_tk01', 'Zn_drop'])
df_H2STK01_v_zndrop = df_H2STK01_v_zndrop.set_index('H2S_tk01')

# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# Create the dataframe for H2S_tk02 v. Zn drop

gas_tk02_start = round(DF_test2['H2S_tk02'][0]-20, 2)
gas_tk02_end = round(DF_test2['H2S_tk02'][0]+20, 2)

gas_2_range = np.linspace(gas_tk02_start, gas_tk02_end, 40)


zn_drop_pred_h2s2 = []
x_list_h2s2 = []
for gas2 in gas_2_range:
    DF_test2['H2S_tk02'] = gas2
    normed_DF_test2 = norm(DF_test2)
    predictions2 = new_model2.predict(normed_DF_test2)
    zn_drop_pred_h2s2.append(predictions2[0][0])
    x_list_h2s2.append(gas2)
    

    
df_H2STK02_v_zndrop = pd.DataFrame(zip(x_list_h2s2, zn_drop_pred_h2s2), columns=['H2S_tk02', 'Zn_drop'])
df_H2STK02_v_zndrop = df_H2STK02_v_zndrop.set_index('H2S_tk02')

In [ ]:
# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# Create the dataframe for varying H2S_tk01 and H2S_tk02 combination and predicting the Zn drop

df_test_columns_Include = ['104_Al', '104_Ca', '104_Co', '104_Cr', '104_Cu', '104_Fe', '104_Fe2',
       '104_Ft', '104_Mg', '104_Mn', '104_NTU', '104_Ni', '104_ORP', '104_Pb',
       '104_Si', '104_Zn', '104_pH60', 'DeZn_Feed_Flow', 'DeZn_Feed_T',
       'DeZn_Seed_Flow', 'DeZn_TK05_T', 'H2S_tk01', 'H2S_tk02', 'tk05_Al',
       'tk05_Ca', 'tk05_Co', 'tk05_Cr', 'tk05_Cu', 'tk05_Fe', 'tk05_Mg',
       'tk05_Mn', 'tk05_Ni', 'tk05_Pb', 'tk05_Si', 'tk05_Zn']

zn_drop_pred_h2s3 = []
#x_list_h2s3 = []
x_list_h2s1_3 =[]
x_list_h2s2_3 =[]
for tk01_gas in gas_1_range:
    for tk02_gas in gas_2_range:
        DF_test2['H2S_tk01'] = tk01_gas
        DF_test2['H2S_tk02'] = tk02_gas
        
        normed_DF_test2 = norm(DF_test2)
        predictions2 = new_model2.predict(normed_DF_test2)
        zn_drop_pred_h2s3.append(predictions2[0][0])
        x_list_h2s1_3.append(tk01_gas)
        x_list_h2s2_3.append(tk02_gas)
    
df_H2STK01_v_zndrop_fac = pd.DataFrame(zip(x_list_h2s1_3, zn_drop_pred_h2s3), columns=['H2S_tk01', 'Zn_drop'])
df_H2STK02_v_zndrop_fac = pd.DataFrame(zip(x_list_h2s2_3, zn_drop_pred_h2s3), columns=['H2S_tk02', 'Zn_drop'])



df_H2S_tk01_02 = pd.merge(df_H2STK01_v_zndrop_fac, df_H2STK02_v_zndrop_fac, on="Zn_drop") #df containing zndrop, h2s_tk01, h2s_tk02


# Create a df containing the H2S flowrate to tk01 and tk02 and temperature of tk05 for the maximum zn_drop and minimum zn drop

max_zndrop = df_H2S_tk01_02[df_H2S_tk01_02['Zn_drop'] == df_H2S_tk01_02['Zn_drop'].max()]
min_zndrop = df_H2S_tk01_02[df_H2S_tk01_02['Zn_drop'] == df_H2S_tk01_02['Zn_drop'].min()]

# Compute the latest H2S factor

Total_H2S_105 = DF_TF['H2S_tk01'][-1]+DF_TF['H2S_tk02'][-1]
H2S_factor = Total_H2S_105/(DF_TF['DeZn_Feed_Flow'][-1]*DF_TF['104_Zn'][-1:]*22.4/65)

# Compute the H2S factor for the minimum and maximum zn drop
#
#

H2S_tk01_min = min_zndrop['H2S_tk01'].iloc[0]
H2S_tk02_min = min_zndrop['H2S_tk02'].iloc[0]
Total_H2S_105_min = H2S_tk01_min + H2S_tk02_min

H2S_factor_min = Total_H2S_105_min/(DF_TF['DeZn_Feed_Flow'][-1]*DF_TF['104_Zn'][-1:]*22.4/65) # the H2S factor at the minimum zn drop
Fraction_H2S_tk01_min = H2S_tk01_min/Total_H2S_105_min
Fraction_H2S_tk02_min = H2S_tk02_min/Total_H2S_105_min

recomm_text1 = '<b>{:.2f}</b> : H2S Factor at min_Zn_drop (<b>{:.2f}</b>) <br> H2S Ratio (tk01:tk02): {:.2f}:{:.2f}'.format(H2S_factor_min.iloc[0], min_zndrop['Zn_drop'].iloc[0],Fraction_H2S_tk01_min, Fraction_H2S_tk02_min)

# Compute the H2S factor for the minimum and maximum zn drop

H2S_tk01_max = max_zndrop['H2S_tk01'].iloc[0]
H2S_tk02_max = max_zndrop['H2S_tk02'].iloc[0]
Total_H2S_105_max = H2S_tk01_max + H2S_tk02_max

H2S_factor_max = Total_H2S_105_max/(DF_TF['DeZn_Feed_Flow'][-1]*DF_TF['104_Zn'][-1:]*22.4/65)
Fraction_H2S_tk01_max = H2S_tk01_max/Total_H2S_105_max
Fraction_H2S_tk02_max = H2S_tk02_max/Total_H2S_105_max

H2S_factor_max = Total_H2S_105_max/(DF_TF['DeZn_Feed_Flow'][-1]*DF_TF['104_Zn'][-1:]*22.4/65) # the H2S factor at the maximum zn drop

recomm_text2 = '<b>{:.2f}</b> : H2S Factor at max_Zn_drop (<b>{:.2f}</b>) <br> H2S Ratio (tk01:tk02): {:.2f}:{:.2f}'.format(H2S_factor_max.iloc[0], max_zndrop['Zn_drop'].iloc[0], Fraction_H2S_tk01_max, Fraction_H2S_tk02_max)

##### Create the Plot

In [ ]:
# Create the plot for the Zn drop predictions and the sensitivity to tk05 temp and H2Stk01 and tk02 flow
#
# DF prerequisite: DF, DF_TF

figzndrop_axis_truefalse_list = [False, False, False]
subplotTitle = ['Zn Drop','Temperature Sensitivity', 'H2S_TK01 Flowrate Sensitivity', 
                     'H2S_TK02 Flowrate Sensitivity',"","","",""]



figzndrop_PRED_TF = make_subplots(
    rows=2, cols=4,
    specs=[[{"rowspan": 1, "colspan": 4},None, None, None], [{"type": "scatter"}, {"type": "scatter"},  {"type": "scatter"}, None]],
    subplot_titles=(subplotTitle),

)


figzndrop_axis_truefalse_list = [False]
generate_plot_from_df(figzndrop_PRED_TF, DF[['Zn_drop_adj',]], figzndrop_axis_truefalse_list)

figzndrop_PRED_TF.add_trace(go.Scatter(x=DF_TF.index, y=DF_TF['TF_pred2'], name='TF_pred2'), secondary_y=False,) #This code adds the TF_pred2 to the plot

# Mark the datetime when there were problems with the pidata
figzndrop_PRED_TF.add_shape(
        # filled Rectangle
            type="rect",
            x0='2020-08-17 23:00:00',
            y0=-1000,
            x1='2020-08-19 13:00:00',
            y1=200,
            fillcolor="LightSalmon",
            opacity=0.5,
            layer="below",
            line_width=0,
        )
figzndrop_PRED_TF.add_shape(
        # filled Rectangle
            type="rect",
            x0='2020-08-26 16:00:00',
            y0=-1000,
            x1='2020-08-28 18:00:00',
            y1=200,
            fillcolor="LightSalmon",
            opacity=0.5,
            layer="below",
            line_width=0,
        )

figzndrop_PRED_TF.add_shape(
        # filled Rectangle
            type="rect",
            x0='2020-08-26 03:00:00',
            y0=-1000,
            x1='2020-08-26 06:00:00',
            y1=200,
            fillcolor="LightSalmon",
            opacity=0.5,
            layer="below",
            line_width=0,
        )

figzndrop_PRED_TF.update_shapes(dict(xref='x', yref='y'))

#figzndrop_PRED.add_trace(go.Scatter(x=DF.loc['2020-08-17 23:00:00':'2020-08-19 13:00:00'], y=DF['ZnDrop_pred'], name='ZnDrop_pred'), secondary_y=True,)


title_plotZNDROP = "<b>Zn Drop Pred Mach 2 Beta</b>" + '<br>'+ "Bottom subplots show zn drop sensitivity to 105TK05 temp, H2S to TK01, H2S to TK02" + '<br>' + 'Updated: {}'.format(datetime.datetime.now().strftime("%m/%d/%Y %H:%M:%S")) + "<br>" + ""
figzndrop_PRED_TF.update_layout(title_text=title_plotZNDROP,hovermode="x",
                       hoverdistance=100, # Distance to show hover label of data point
                       spikedistance=1000, # Distance to show spike
                       xaxis=dict(linecolor="#BCCCDC",
                                   showspikes=True, # Show spike line for X-axis
                                   # Format spike
                                   spikethickness=2,
                                   spikedash="dot",
                                   spikecolor="#999999",
                                   spikemode="across"), font=dict(size=10)
                               )
# Adds figure title
figzndrop_PRED_TF.update_xaxes(title_text="DateTime", row=1, col=1) # Set x-axis title
figzndrop_PRED_TF.update_yaxes(title_text="Zndrop", row=1, col=1, secondary_y=False) # Set y-axes titles primary axis

#
#       
figzndrop_PRED_TF.add_trace(go.Scatter(x=df_tk05temp_v_zndrop.index,
                             y=df_tk05temp_v_zndrop['Zn_drop'].dropna(how='any'),
                             name='Temp Sensitivity'),row=2, col=1)
figzndrop_PRED_TF.add_trace(go.Scatter(x=df_H2STK01_v_zndrop.index,
                             y=df_H2STK01_v_zndrop['Zn_drop'].dropna(how='any'),
                             name='H2S_TK01 Sensitivity'),row=2, col=2)
figzndrop_PRED_TF.add_trace(go.Scatter(x=df_H2STK02_v_zndrop.index,
                             y=df_H2STK02_v_zndrop['Zn_drop'].dropna(how='any'),
                             name='H2S_TK02 Sensitivity'),row=2, col=3)

# Adds figure title
figzndrop_PRED_TF.update_xaxes(title_text="DeZn_TK05_T (deg Celcius)", row=2, col=1)# Set x-axis title
figzndrop_PRED_TF.update_yaxes(title_text="Zndrop (%)", row=2, col=1, secondary_y=False) # Set y-axes titles primary axis

figzndrop_PRED_TF.update_xaxes(title_text="H2S_TK01_Flow (m3/h) ",  row=2, col=2) # Set x-axis title
figzndrop_PRED_TF.update_yaxes(title_text="Zndrop (%)", row=2, col=2, secondary_y=False) # Set y-axes titles primary axis

figzndrop_PRED_TF.update_xaxes(title_text="H2S_TK02_Flow (m3/h)", row=2, col=3) # Set x-axis title
figzndrop_PRED_TF.update_yaxes(title_text="Zndrop (%)", row=2, col=3, secondary_y=False) # Set y-axes titles primary axis

figzndrop_PRED_TF.update_layout(annotations=[
            go.layout.Annotation(
                text='Latest H2S Factor: <b>{:.2f}</b>'.format(H2S_factor[0])+ '<br><br>' + recomm_text1+'<br><br>'+recomm_text2+'<br>'+
                '<br> For the PLS on {}'.format(DF_test2.index[-1].strftime("%m/%d/%Y %H:%M:%S"))+"<br>",
                align='left',
                showarrow=False,
                xref='paper',
                yref='paper',
                x=.9,
                y=.09,
                #width=300,
                #height=250,
                #bgcolor= "rgba(1,1,1,0.2)",
                bordercolor='gray',
                borderwidth=0.5,
                font=dict(
        size=12)                 
            )
        ]
)

# -------------------------------------------------------------------------------------------------------------Code for adding and marking the plot with the latest value-------------------------------------------------------------------

# arguments latest_val is the last good value in the dataset, df is the main dataset for the combined Pi and Laboratory Results
def insert_temp_to_dfrange_get_index(latest_val, df):
    for i in range(0, len(df.index)):
        if (latest_val >= df.index[i]) & (latest_val <= df.index[i+1]):
            xi = i
            break;
        else:
            pass
    return xi

latest_temp_i = DF['DeZn_TK05_T'][-1]
xi = insert_temp_to_dfrange_get_index(latest_temp_i, df_tk05temp_v_zndrop)

latest_tk01_i = DF['H2S_tk01'][-1]
x_tk01_i = insert_temp_to_dfrange_get_index(latest_tk01_i, df_H2STK01_v_zndrop)

latest_tk02_i = DF['H2S_tk02'][-1]
x_tk02_i = insert_temp_to_dfrange_get_index(latest_tk02_i, df_H2STK02_v_zndrop)

figzndrop_PRED_TF.add_trace(go.Scatter(x=[df_tk05temp_v_zndrop.index[xi]],
                             y=[df_tk05temp_v_zndrop['Zn_drop'].iloc[xi]],
                             name='Latest DeZn TK05 Temp', marker=dict(size=10)),row=2, col=1)

figzndrop_PRED_TF.add_trace(go.Scatter(x=[df_H2STK01_v_zndrop.index[x_tk01_i]],
                             y=[df_H2STK01_v_zndrop['Zn_drop'].iloc[x_tk01_i]],
                             name='Latest H2S_TK01_flow', marker=dict(size=10)),row=2, col=2)

figzndrop_PRED_TF.add_trace(go.Scatter(x=[df_H2STK02_v_zndrop.index[x_tk02_i]],
                             y=[df_H2STK02_v_zndrop['Zn_drop'].iloc[x_tk02_i]],
                             name='Latest H2S_TK02_flow', marker=dict(size=10)),row=2, col=3)

# -------------------------------------------------------------------------------------------------------------End---------------------------------------------------------------------------------------------------

In [ ]:
# Create the plot for the Zn drop predictions and the sensitivity to tk05 temp and H2Stk01 and tk02 flow
#
# DF prerequisite: DF, DF_TF

#figznNi_axis_truefalse_list = [False, False, True, True]
figznNi_PRED_TF = make_subplots(
    rows=1, cols=1,
    specs=[[{"type": "scatter"}]],
)

#generate_plot_from_df(figzn_PRED_TF, DF[['106_Zn']], True)

figzn_PRED_TF.add_trace(go.Scatter(x=DF_TF.index, y=DF_TF['MSTK01_Zn_Pred'], name='MSTK01_Zn_Pred'), secondary_y=True,) #This code adds the TF_pred2 to the plot
figzn_PRED_TF.add_trace(go.Scatter(x=DF_TF.index, y=DF_TF['MSTK01_Ni_Pred'], name='MSTK01_Ni_Pred'), secondary_y=False,) #This code adds the TF_pred2 to the plot

figzn_PRED_TF.add_trace(go.Scatter(x=DF.index, y=DF['106_Zn'], name='106_Zn'), secondary_y=False,) #This code adds the TF_pred2 to the plot
figzn_PRED_TF.add_trace(go.Scatter(x=DF.index, y=DF['106_Ni'], name='106_Ni'), secondary_y=False,) #This code adds the TF_pred2 to the plot

In [ ]:
#-------------------------------------------------------------------------End----------------------------------------------------------------------------
plt.close()
with open(r'C:\Users\v.t.flores\Documents\Zndrop_pred_mach2_b.html', 'w') as f:
    f.write(figzndrop_PRED_TF.to_html(figzndrop_PRED_TF, full_html=True, include_plotlyjs='cdn'))
    f.write(figzn_PRED_TF.to_html(figzn_PRED_TF, full_html=True, include_plotlyjs='cdn'))
    
#-------------------------------------------------------------------------End----------------------------------------------------------------------------
plt.close()
with open(r'\\THGOBNA002\thpal\05 Production\150 DCS\Miscellaneous\Zndrop_pred_mach2_b.html', 'w') as f:
    f.write(figzndrop_PRED_TF.to_html(figzndrop_PRED_TF, full_html=True, include_plotlyjs='cdn'))
    f.write(figzn_PRED_TF.to_html(figzn_PRED_TF, full_html=True, include_plotlyjs='cdn'))
       